In [1]:
import scipp as sc
import scippneutron as scn
import numpy as np
import h5py as h5
from shutil import copyfile
import sys
from ipyfilechooser import FileChooser

In [2]:
def fix_nexus_file(infile, outfile):
    """
    Currently there are some tweaks to make file working in scipp. This function won't be necessary in the future
    """

    copyfile(infile, outfile)
    with h5.File(outfile, 'r+') as f:
        
        #For loading in Mantid instrument name, sample position and detector_ids for monitors are needed
        f['entry/instrument/name'] = 'LARMOR'
        
        group_entry = f['entry']
        nx_class = group_entry.create_group('sample')
        nx_class.attrs["NX_class"] = 'NXsample'
       
        ##Adding detector_id
        #f['entry']['instrument']['monitor_1']['detector_number'] = np.array(458753)
        #f['entry']['instrument']['monitor_2']['detector_number'] = np.array(458754)
        group = f['entry/instrument']
        for monitor_name in filter(lambda k: k.startswith('monitor'), group):
            monitor_group = group[monitor_name]
            monitor_event_group = monitor_group[f'{monitor_name}_events']
            for key in list(monitor_event_group):
                monitor_group[key] = monitor_event_group.pop(key)
            del monitor_group[f'{monitor_name}_events']


In [3]:
# Create and display a FileChooser widget
fc = FileChooser('/Users/wojciechpotrzebowski/SCIPP_SPACE/loki_detector_test/ess/docs/instruments/loki/Larmor_data')
display(fc)

FileChooser(path='/Users/wojciechpotrzebowski/SCIPP_SPACE/loki_detector_test/ess/docs/instruments/loki/Larmor_…

### Load and show data

In [52]:
input_file = fc.selected
fixed_file = f'{input_file[:-4]}_fixed.nxs'

In [53]:
fix_nexus_file(input_file, fixed_file)

In [54]:
data = scn.load_nexus(data_file=fixed_file)

/Users/wojciechpotrzebowski/opt/anaconda3/envs/ess_0.4/lib/python3.8/site-packages/scippneutron/file_loading/_monitor_data.py:66: UserWarning: Event data present in NXmonitor group /entry/instrument/monitor_1. Histogram-mode monitor data from this group will be ignored.
  warnings.warn(f"Event data present in NXmonitor group {group.name}. "
/Users/wojciechpotrzebowski/opt/anaconda3/envs/ess_0.4/lib/python3.8/site-packages/scippneutron/file_loading/_monitor_data.py:66: UserWarning: Event data present in NXmonitor group /entry/instrument/monitor_2. Histogram-mode monitor data from this group will be ignored.
  warnings.warn(f"Event data present in NXmonitor group {group.name}. "


### Plot data and show how many events and pixels do we have

In [55]:
data

<scipp.DataArray>
Dimensions: Sizes[detector_id:458752, tof:1, ]
Coordinates:
  detector_id                 int32           [None]  (detector_id)  [1, 2, ..., 458751, 458752]
  position                  vector3              [m]  (detector_id)  [(-0.499023, 0.45326, 4.42333), (-0.497074, 0.45326, 4.42333), ..., (0.495061, -0.4437, 4.51745), (0.49701, -0.4437, 4.51745)]
  sample_position           vector3              [m]  ()  [(0, 0, 0)]
  source_position           vector3              [m]  ()  [(0, 0, -25.3)]
  tof                         int32             [ns]  (tof [bin-edge])  [56, 99999728]
Data:
                          DataArrayView           [None]  (detector_id, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                         int32             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                         int32             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, ..., <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                         int32             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                         int32             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

]
Attributes:
  base_position             vector3              [m]  (detector_id)  [(0, -0.00114, -0.00767), (0.00194919, -0.00114, -0.00767), ..., (0.994085, -0.8981, 0.086447), (0.996034, -0.8981, 0.086447)]
  end_time                   string           [None]  ()  ["2032-03-02T10:17:12.656Z"]
  experiment_title           string           [None]  ()  ["My experiment"]
  instrument_name            string           [None]  ()  ["LARMOR"]
  monitor_1               DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[detector_id:1, tof:1, ]
Coordinates:
  detector_id                 int32           [None]  (detector_id)  [1]
  tof                         int32             [ns]  (tof [bin-edge])  [23679, 99998968]
Data:
                          DataArrayView           [None]  (detector_id, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:2761297, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  [2022-03-02T22:17:12.106361659, 2022-03-02T22:17:12.106361659, ..., 2022-03-02T22:25:02.506346157, 2022-03-02T22:25:02.506346157]
  tof                         int32             [ns]  (event)  [11346719, 11364868, ..., 31172130, 31738849]
Data:
                            float32         [counts]  (event)  [1, 1, ..., 1, 1]  [1, 1, ..., 1, 1]

]

]
  monitor_2               DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[detector_id:1, tof:1, ]
Coordinates:
  detector_id                 int32           [None]  (detector_id)  [2]
  tof                         int32             [ns]  (tof [bin-edge])  [4271, 99999706]
Data:
                          DataArrayView           [None]  (detector_id, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:11181303, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  [2022-03-02T22:17:12.106361659, 2022-03-02T22:17:12.106361659, ..., 2022-03-02T22:25:02.506346157, 2022-03-02T22:25:02.506346157]
  tof                         int32             [ns]  (event)  [11363198, 11478845, ..., 85878402, 86371028]
Data:
                            float32         [counts]  (event)  [1, 1, ..., 1, 1]  [1, 1, ..., 1, 1]

]

]
  position_transformations Variable           [None]  ()  

In [56]:
#How many events?
data.bins.constituents['data']

<scipp.DataArray>
Dimensions: Sizes[event:383638, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  [2022-03-02T22:23:16.906349291, 2022-03-02T22:23:54.806348212, ..., 2022-03-02T22:23:10.306349655, 2022-03-02T22:19:16.006357286]
  tof                         int32             [ns]  (event)  [47054019, 50937305, ..., 21730320, 10313995]
Data:
                            float32         [counts]  (event)  [1, 1, ..., 1, 1]  [1, 1, ..., 1, 1]

In [57]:
len(data.coords['position'])

458752

In [58]:
data.plot()

### Instrument view

In [59]:
scn.instrument_view(data, pixel_size=0.01)

### Histograming data

In [60]:
tof_edges = sc.linspace(dim='tof', start=15, stop=99999956, num=100, unit='ns')
histoigramed = sc.histogram(data, bins=tof_edges)

In [61]:
scn.instrument_view(histoigramed, pixel_size=0.01)

## Sorting out monitors

In [62]:
monitors = {
    'sample': {'incident': data.attrs["monitor_1"].value,
               'transmission': data.attrs["monitor_2"].value}
}

In [63]:
monitors['sample']['incident'].coords['position'] = sc.vector(value = np.array([0, 0, 0.0]), unit = sc.units.m)
monitors['sample']['incident'].coords['source_position'] = sc.vector(value = np.array([0, 0, -25.3]), unit = sc.units.m)
monitors['sample']['transmission'].coords['position'] = sc.vector(value = np.array([0, 0, 0.0]), unit = sc.units.m)
monitors['sample']['transmission'].coords['source_position'] = sc.vector(value = np.array([0, 0, -25.3]), unit = sc.units.m)

In [64]:
tof_edges = sc.linspace(dim='tof', start=7291, stop=99999036, num=100, unit='ns')
histogrammed_monitor1 = sc.histogram(monitors['sample']['incident'], bins=tof_edges)
histogrammed_monitor2 = sc.histogram(monitors['sample']['transmission'], bins=tof_edges)

In [65]:
sc.plot({'data': histoigramed.sum('detector_id'), 'monitor1':histogrammed_monitor1.sum('detector_id'), 
        'monitor2':histogrammed_monitor2.sum('detector_id')})

### Troubleshooting

In [66]:
#path = 'Larmor_data'
#scn.load(filename=f'{path}/2022-02-23_1100_detector_id.nxs', mantid_alg='LoadEventNexus')

In [67]:
binned_data = data.bins.sum()
scn.instrument_view(binned_data)
binned_data.plot()
binned_data
binned_data.sum('detector_id').plot(marker='.')

/Users/wojciechpotrzebowski/opt/anaconda3/envs/ess_0.4/lib/python3.8/site-packages/scipp/plotting/figure1d.py:339: UserWarning: Attempting to set identical bottom == top == 383638.0 results in singular transformations; automatically expanding.
  self.ax.set_ylim(vmin, vmax)
